## Particle in Cell method for the one-dimensional two-counterstreaming plasma

In [ ]:
import numpy as np
import scipy.linalg as la

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
def twoStreamPICsim(length=1.,
                    timeStep=0.1,
                    initStreamVelocity=0.1,
                    posPerturb_stddev=1/50,
                    velPerturb_stddev=.1/50,
                    qmRatio=.1,
                    numSteps=100, 
                    numCells=50, 
                    numParticles=100,
                    color1='r',
                    color2='b',
                    displayOrSave=True,
                    fps=25):
    cellSize = length / numCells
    fixedChargedDensity = numParticles * qmRatio / length
    
    # Decleare and pre-fill with zeros charge densities, potentials, el. fields in cells
    potentials = np.zeros((numSteps, numCells))
    elfields = np.zeros((numSteps, numCells))
    chargedensities = np.full((numSteps, numCells), fixedChargedDensity)
    
    # Decleare and pre-fill particles positions and velocities, and add small perturbations
    positions = np.zeros((numSteps, numParticles))
    positions[0] = np.linspace(0, length, numParticles)
    positions[0] += np.random.randn(numParticles) * posPerturb_stddev
    velocities = np.zeros((numSteps, numParticles))
    velocities[0] = ((np.arange(numParticles)%2)*2-1)*initStreamVelocity
    velocities[0] += np.random.randn(numParticles) * velPerturb_stddev
    
    # Call the function that runs the main simulation loop
    runSimulation(positions, velocities, potentials, elfields, chargedensities,
                  length, timeStep, cellSize, qmRatio, numSteps, numCells, numParticles)
    
    # Calculate total momentum and energies for all simulation time steps
    totalMomentum = np.sum(velocities, axis=1)
    totalKinEnergy = 0.5*np.sum(velocities**2, axis=1)
    totalPotEnergy = 0.5*cellSize*np.sum(elfields**2, axis=1)
    totalEnergy = totalKinEnergy + totalPotEnergy
    
    # Visualize the total momentum and energy evolution over time
    plt.plot(totalMomentum)
    plt.title("Total momentum of the system over time")
    
    plt.figure()
    plt.plot(totalEnergy)
    plt.title("Total energy of the system over time")
    
    # Call the function that creates an interactive animation of the simulation
    animateSimulation(positions, velocities, numParticles, length, initStreamVelocity, 
                      color1, color2, displayOrSave, fps, numSteps)

In [ ]:
def runSimulation(positions, velocities, 
                  potentials, elfields, chargedensities,
                  length, timeStep, cellSize, qmRatio, 
                  numSteps, numCells, numParticles):
    for it in range(numSteps-1):
        positions[it] %=  length

        # Assign charge densities to cells (using nearest-neighbour approach)
        nearestCells = np.array(np.round((positions[it] - cellSize/2)/ cellSize), dtype=int) % numCells
        np.add.at(chargedensities[it], nearestCells, -1)
        chargedensities[it] /= cellSize

        # Calculate the potentials
        potSolverMatrix = (np.diag(np.full(numCells,-2.))
                           + np.diag(np.ones(numCells-1),1)
                           + np.diag(np.ones(numCells-1),-1))                
    #     potSolverMatrix[0,-1]=1
    #     potSolverMatrix[-1,0]=1
        potentials[it] = la.solve(potSolverMatrix, -chargedensities[it]*(cellSize)**2)

        # Calculate the electric fields
        elfieldSolver = np.concatenate([[potentials[it,-1]], potentials[it], [potentials[it,0]]])
        elfields[it] = ((-0.5/cellSize)
                        * (elfieldSolver[2:2+numCells]-elfieldSolver[:numCells]))
        # Calculate the electric fields at particle positions
        elfields_atparticles = elfields[it, nearestCells]

        # Update velocities and positions (based on electric fields)
        velocities[it+1] = velocities[it] + timeStep*qmRatio*elfields_atparticles
        positions[it+1] = positions[it] + timeStep*velocities[it+1]

In [ ]:
def animateSimulation(positions, velocities, numParticles, length, initStreamVelocity, 
                      color1, color2, displayOrSave, fps, numSteps):
    mpl.rcParams.update(mpl.rcParamsDefault)
    mpl.rcParams['animation.embed_limit'] = 2**64

    fig, ax = plt.subplots()
    colors = np.array([(color1 if i%2==0 else color2) for i in range(numParticles)])
    scatterPlot = ax.scatter(0, 0, s=4.)

    # Set up axis limits for both static and dynamic cases
    ax.set_xlim(0, length)
    ax.set_ylim(-initStreamVelocity*5, initStreamVelocity*5)
    dynamicAxisLimits = False
    axisLimitWspace = 1.2
    minVelocities = np.min(velocities, axis=1) * axisLimitWspace
    minVelocities = minVelocities * (minVelocities<0)
    maxVelocities = np.max(velocities, axis=1) * axisLimitWspace
    maxVelocities = maxVelocities * (maxVelocities>0)
    
    # Function that gets called at each frame to update the figure
    def animateStep(frame):
        scatterPlot.set_offsets(np.vstack([positions[frame],velocities[frame]]).T)
        scatterPlot.set_color(colors)
        
        # Allow for dynamic, on-the-fly resizing of the figure
        if dynamicAxisLimits and frame%10==0:
            ax.set_ylim(minVelocities[frame], maxVelocities[frame])
            
        return scatterPlot,
    
    positions+=length/2
    positions%=length
    anim = animation.FuncAnimation(fig, 
                         func=animateStep, 
                         frames=np.arange(0,numSteps,2), 
                         interval=1000/fps,
                         blit=True)
    
    # Allow for either immediately displaying the simulation, or saving it as .mp4 file
    if displayOrSave:
        display(HTML(anim.to_jshtml()))
    else:       
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
        anim.save('TwoStreams_PICsimulation.mp4', writer=writer)

In [ ]:
twoStreamPICsim(length=1.,
                timeStep=0.01,
                initStreamVelocity=0.3,
                posPerturb_stddev=0.03,
                velPerturb_stddev=0,
                qmRatio=1e-4,
                numSteps=1000, 
                numCells=80, 
                numParticles=500,
                color1='r',
                color2='b',
                displayOrSave=True,
                fps=35)